In [1]:
import os
import logging
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [3]:
import time
import logging
import json
import numpy as np
import pandas as pd
from typing import Dict, Any, List, Optional, Union, Iterator
from opensearch_toolkits.core import SyncOpenSearchManagerSearcher, OpenSearchConfig

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [7]:
searcher = SyncOpenSearchManagerSearcher(
    OpenSearchConfig(
            host=os.environ["OPENSEARCH_HOST"],  # OpenSearch 서버 주소
            port=int(os.environ["OPENSEARCH_PORT"]),  # OpenSearch 서버 포트
            user=os.environ.get("OPENSEARCH_USER"),  # OpenSearch 사용자 이름
            password=os.environ.get("OPENSEARCH_PASSWORD"),  # OpenSearch 비밀번호
            use_ssl=os.environ["OPENSEARCH_USE_SSL"].lower() == "true",  # SSL 사용 여부
    )
)
index_name = "korquad_v1"

In [8]:
def basic_search_example():
    """기본 검색 예제"""
    print("\n===== 기본 검색 예제 =====")
    
    # match_all 쿼리
    query = {"match_all": {}}
    
    try:
        result = searcher.search(index_name, query)
        print(f"검색 결과: {result['total_hits']}개 문서 발견")
        print(f"검색 시간: {result['took']}ms (클라이언트 측: {result['client_took_ms']}ms)")
        
        # 첫 번째 결과만 출력
        if result['hits']:
            print("\n첫 번째 문서:")
            print(json.dumps(result['hits'][0]['_source'], indent=2, ensure_ascii=False))
    except Exception as e:
        print(f"검색 중 오류 발생: {str(e)}")

In [10]:
basic_search_example()

INFO:opensearch:HEAD http://192.168.219.104:9200/korquad_v1 [status:200 request:0.018s]
INFO:opensearch:POST http://192.168.219.104:9200/korquad_v1/_search [status:200 request:0.088s]



===== 기본 검색 예제 =====
검색 결과: 1000개 문서 발견
검색 시간: 2ms (클라이언트 측: 100ms)

첫 번째 문서:
{
  "metadata": {
    "title": "파우스트_서곡"
  },
  "type": "Document",
  "text": "바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?",
  "vector_field": [
    -0.01710040308535099,
    0.0008359348867088556,
    -0.029803559184074402,
    0.02784922905266285,
    -0.02797137387096882,
    -0.03420080617070198,
    0.04323958978056908,
    -0.05887424573302269,
    0.03957521915435791,
    0.010932043194770813,
    -0.014901779592037201,
    0.026505624875426292,
    -0.04617108777165413,
    -0.030658580362796783,
    0.019543318077921867,
    0.015878945589065552,
    0.04836971312761307,
    -0.03444509953260422,
    0.0073898169212043285,
    -0.039086636155843735,
    0.038842346519231796,
    -0.057164207100868225,
    0.025894897058606148,
    0.05032404512166977,
    -0.02479558438062668,
    0.03444509953260422,
    -0.019421173259615898,
    -0.03224647417664528,
    0.001923795323818922,
    -0.008305910043418407,
    0.

In [18]:
# 예제 4: 쿼리로 검색
def query_search_example():
    """다양한 쿼리 검색 예제"""
    print("\n===== 다양한 쿼리 검색 예제 =====")
    
    # 예제 1: 텍스트 필드 검색
    print("\n1. 텍스트 필드 검색:")
    query = {
        "match": {
            "text": "파우스트"
        }
    }
    result = searcher.search(index_name, query)
    print(f"'sample' 단어 검색 결과: {result['total_hits']}개 문서 발견")
    
    # 예제 2: 한글 텍스트 검색
    print("\n2. 한글 텍스트 검색:")
    query = {
        "match": {
            "text.korean": "역사"
        }
    }
    result = searcher.search(index_name, query)
    print(f"'예제' 단어 검색 결과: {result['total_hits']}개 문서 발견")
    
    # 예제 4: bool 쿼리 조합
    print("\n4. Bool 쿼리 조합:")
    query = {
        "bool": {
            "must": [
                {"match": {"text": "역사"}},
            ]
        }
    }
    result = searcher.search(index_name, query)
    print(f"'sample' 포함 AND 'article' 타입 검색 결과: {result['total_hits']}개 문서 발견")
